In [5]:
import pandas as pd
import numpy as np

# import data

This method was designed on a dataset where the columns represent genes and the rows represent cell types. Values are the log2 values of the counts per median (CPM) used to normalize the expression data across the cells to account for read depth differences between the cells. These data have already been filtered to remove cells that did not pass thresholds for read counts and other technical measurements.

In [2]:
filename = 'single_cell_data.tsv.gz'
data = pd.read_csv(filename, sep='\t', index_col='index')
data = data.loc[:,data.columns.str.contains('TGGT1')] #data inclues ERCC (RNA standards) that do not need be considered for this part of the analysis

# Standard deviation analysis

Let's say you have a gene that is normally highly expressed accross the cells in your dataset and in one cell, that gene is not detected and you want to ask: are there other genes that are aberrantly expressed in that cell and, if so, what are they? 

This approach allows for that. First, for each gene, the standard deviation is calculated and used to calculate the acceptable range for a gene's expression in a cell in the dataset. The user inputs a multiplier of the standard deviation to use in the creation of this range, a larger number means a wider range of non-outlier values. With this range, each gene's expression each cell is determined to be either above the range (a), below the range (b), or within the range (w). A dataframe is outputed with these values for all the data, as well as two other dataframes that count the numbers of a, b, and w values for each gene (across the cells) or each cell (across the genes). Together these outputs allow one to determine if a particular cell has a number of outier genes. 

In [3]:
def standardDeviationAnalysis(data, std_multiple, columnCounts=False, rowCounts=False):
    '''input is a single-cell RNA sequencing data where columns are genes and rows are cells
        using the provided value, determine the range of expression for each gene and then evaluate 
        if a cell is expressing that gene above (a), below (b), or within that range (w)'''
    mask_high = data > (data.mean() + data.std() * std_multiple)
    mask_low = data < (data.mean() - data.std() * std_multiple)
    mask_in_range = ((data <= (data.mean() + (data.std() * std_multiple))) & (data >= (data.mean() - (data.std() * std_multiple))))
    output = pd.DataFrame().reindex_like(data) # creates empty dataframe with data structure of columns and rows
    output.replace(np.nan, '', inplace=True) # replaces the default of np.nan with empty strings
    output.mask(mask_high, other='a', inplace=True) 
    output.mask(mask_low, other='b', inplace=True)
    output.mask(mask_in_range, other='w', inplace=True)
    outputCountsByColumns = output.apply(lambda x:x.value_counts()).T.replace(np.nan,0) # value counts for the values of all the rows for a col
    outputCountsByRows = output.T.apply(lambda x:x.value_counts()).T.replace(np.nan,0) # values counts for the values of all columns for a row 
    return(output, outputCountsByColumns, outputCountsByRows)


def outlierLists(data, classier='w'):
    df = data.T
    dic = {}
    for i in list(df.columns):
        outliers = df[i].loc[(df[i] != classier)].index
        count = len(outliers)
        dic[i] = list([count, list(outliers)])
    output = pd.DataFrame.from_dict(dic, orient='index', columns=['count', 'outliers'])
    return(output)


In [7]:
outlierData, outliersCountsByGene, outliersCountsByCell = standardDeviationAnalysis(data, 10, columnCounts=False, rowCounts=False)
outliersInCells = outlierLists(outlierData)